In [1]:
import numpy as np 
import pandas as pd 


In [2]:
import tensorflow as tf
from tensorflow import keras


In [3]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


From https://www.tensorflow.org/api_docs/python/tf/nn/conv2d

Given an input tensor of shape batch_shape + [in_height, in_width, in_channels] and a filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels], this op performs the following:

Flattens the filter to a 2-D matrix with shape [filter_height * filter_width * in_channels, output_channels].

Extracts image patches from the input tensor to form a virtual tensor of shape [batch, out_height, out_width, filter_height * filter_width * in_channels].

For each patch, right-multiplies the filter matrix and the image patch vector.

In [13]:
class spectralConv2D(keras.layers.Layer):
    
    def __init__(self, filters = 32,kernel_size=(3,3),strides=(1, 1), padding='valid',activation=tf.nn.relu):
        
        # Initialization
        super(spectralConv2D, self).__init__()
        assert len(kernel_size) > 1 , "Please input the Kernel Size as a 2D tuple"
        self.strides = strides 
        self.padding = padding
        self.filters = filters
        self.kernel_size = kernel_size
        self.activation = activation
        
    def build(self, input_shape):
        # Initialize Filters 
        
        # Assuming Input_shape is channels_last 
        kernel_shape = [self.kernel_size[0],self.kernel_size[1],input_shape[3],self.filters]
        
        self.kernel_real = self.add_weight( shape=kernel_shape,
                                            initializer="glorot_uniform",
                                            trainable=True)
        self.kernel_imag = self.add_weight( shape=kernel_shape,
                                            initializer="glorot_uniform",
                                            trainable=True)
        
        self.bias = self.add_weight(shape=(self.filters,), initializer="zeros", trainable=True)

    def call(self, inputs):
        
        kernel = tf.complex(self.kernel_real,kernel_imag)
        spatial_kernel = tf.signal.ifft2d(kernel)
        
        print(spatial_kernel.shape)
        
        convolution_output = tf.nn.convolution(
                            inputs,
                            spatial_kernel,
                            strides=list(self.strides),
                            padding=self.padding
                        )
        
        convolution_output = tf.nn.bias_add(convolution_output, self.bias, data_format=self._tf_data_format)
        
        if self.activation is not None:
            convolution_output= self.activation(convolution_output)
            
        return convolution_output